In [3]:
import torch
from sklearn.model_selection import train_test_split

# load data from data folder(we"re in notebooks folder)
sim_arr_tensor = torch.load('./data/sim_arr_tensor.pt')

# train test split
train_data, test_data = train_test_split(
    sim_arr_tensor, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(
    train_data, test_size=0.2, random_state=42)

tensor(1.)
tensor(-1.)


/tmp/ipykernel_600291/642443659.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sim_arr_tensor = torch.load('./data/sim_arr_tensor.pt')


In [4]:
from torch.utils.data import DataLoader

import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import optuna

from models import variational, LitModel
from cluster_acc import adj_rand_index
from search_space import search_space

def objective(trial):
    lr = trial.suggest_float('lr', 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 96, 128, 256])

    num_layers, poolsize, channels, kernel_sizes, dilations, activations = search_space(
        trial, input_dim=3, output_dim=12)
    
    beta = trial.suggest_float('beta', 1, 1000, log=True)
    

    encoder = variational.Encoder(num_layers, poolsize, channels,
                            kernel_sizes, dilations, activations)
    decoder = variational.Decoder(encoder)
    model = variational.DeepEmbedding(encoder, decoder, n_clusters=13)

    lit_model = LitModel(model, lr=lr, beta=beta)

    trainer = L.Trainer(callbacks=[EarlyStopping(monitor="val_loss")])
    trainer.fit(model=lit_model,
                train_dataloaders=DataLoader(train_data, batch_size=batch_size, num_workers=31),
                val_dataloaders=DataLoader(val_data, batch_size=batch_size, num_workers=31))

    labels = model.classify(sim_arr_tensor)
    labels = labels.cpu().detach().numpy()

    return adj_rand_index(labels)
study = optuna.create_study(direction='maximize',
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name='vae_study',
                            storage='sqlite:///vae_study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=100)


[I 2024-08-28 21:55:22,763] A new study created in RDB with name: vae_study
/home/tummalas/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.9/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 2, 2, 2, 5] which is of type list.
  warnings.warn(message)
/home/tummalas/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.9/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 2, 2, 5, 2] which is of type list.
  warnings.warn(message)
/home/tummalas/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.9/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 2, 5, 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2024-08-28 23:08:39,351] Trial 29 failed with parameters: {'lr': 0.000124164779297745, 'batch_size': 32, 'poolsize_5': [2, 5, 2, 2, 2], 'channels_0': 14, 'channels_1': 6, 'channels_2': 2, 'channels_3': 7, 'kernel_0': 64, 'kernel_1': 64, 'kernel_2': 32, 'kernel_3': 8, 'kernel_4': 16, 'dilation_0': 3, 'dilation_1': 3, 'dilation_2': 5, 'dilation_3': 3, 'dilation_4': 5, 'activation_0': 'nn.Tanh', 'activation_1': 'nn.SiLU', 'activation_2': 'nn.SELU', 'activation_3': 'nn.Softplus', 'activation_4': 'nn.SiLU', 'beta': 8.935767472706848} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "/home/tummalas/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py", line 47, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/tummalas/hyper-param-optim-4-vector-field-clustering/.venv/lib/

NameError: name 'exit' is not defined